In [4]:
import os, sys
import numpy as np
import keras.optimizers
import tensorflow as tf
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
import keras.backend as KL
import keras.layers as KL
import keras.models as KM
from keras.utils.training_utils import multi_gpu_model

# Root directory of the project
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '../../'))
sys.path.append(ROOT_DIR)
MODEL_DIR       = os.path.join(ROOT_DIR, 'demo', 'model', 'logs')

from src.parallel_model import ParallelModel

In [2]:
# Load MNIST Data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = np.expand_dims(x_train, -1).astype('float32') / 255
x_test = np.expand_dims(x_test, -1).astype('float32') / 255
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

def build_model(x_train, num_classes):
    # Reset default graph. Keras leaves old ops in the graph,
    # which are ignored for execution but clutter graph
    # visualization in TensorBoard.
    tf.reset_default_graph()

    inputs = KL.Input(shape=x_train.shape[1:], name="input_image")
    x = KL.Conv2D(32, (3, 3), activation='relu', padding="same",
                  name="conv1")(inputs)
    x = KL.Conv2D(64, (3, 3), activation='relu', padding="same",
                  name="conv2")(x)
    x = KL.MaxPooling2D(pool_size=(2, 2), name="pool1")(x)
    x = KL.Flatten(name="flat1")(x)
    x = KL.Dense(128, activation='relu', name="dense1")(x)
    x = KL.Dense(num_classes, activation='softmax', name="dense2")(x)

    return KM.Model(inputs, x, "digit_classifier_model")

x_train shape: (60000, 28, 28, 1)
x_test shape: (10000, 28, 28, 1)


# Type1

In [5]:
# Build data generator and model
datagen = ImageDataGenerator()
model = build_model(x_train, 10)
print ('IP Names     : ', model.input_names)
print ('Model Inputs : ', model.inputs)

# Add multi-GPU support.
type1 = False
GPU_COUNT = 2
if type1:
    model = ParallelModel(model, GPU_COUNT)
else:
    model = multi_gpu_model(model, gpus=GPU_COUNT)
    
optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, clipnorm=5.0)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])

# model.summary()

['input_image']
[<tf.Tensor 'input_image:0' shape=(?, 28, 28, 1) dtype=float32>]


In [6]:
## Train
model.fit_generator(
    datagen.flow(x_train, y_train, batch_size=64),
    steps_per_epoch=500, epochs=10, verbose=1,
    validation_data=(x_test, y_test),
    callbacks=[keras.callbacks.TensorBoard(log_dir=MODEL_DIR,
                                           write_graph=True)]
)

Epoch 1/10
500/500 [==============================] - 8s 15ms/step - loss: 0.3800 - acc: 0.8855 - val_loss: 0.1396 - val_acc: 0.9570
Epoch 2/10
500/500 [==============================] - 3s 7ms/step - loss: 0.1126 - acc: 0.9665 - val_loss: 0.0735 - val_acc: 0.9758
Epoch 3/10
500/500 [==============================] - 3s 7ms/step - loss: 0.0717 - acc: 0.9794 - val_loss: 0.0612 - val_acc: 0.9800
Epoch 4/10
500/500 [==============================] - 3s 7ms/step - loss: 0.0583 - acc: 0.9814 - val_loss: 0.0510 - val_acc: 0.9835
Epoch 5/10
500/500 [==============================] - 3s 7ms/step - loss: 0.0447 - acc: 0.9862 - val_loss: 0.0392 - val_acc: 0.9864
Epoch 6/10
500/500 [==============================] - 3s 7ms/step - loss: 0.0364 - acc: 0.9890 - val_loss: 0.0448 - val_acc: 0.9857
Epoch 7/10
500/500 [==============================] - 3s 7ms/step - loss: 0.0341 - acc: 0.9899 - val_loss: 0.0405 - val_acc: 0.9873
Epoch 8/10
500/500 [==============================] - 3s 7ms/step - loss: 0

# Type2

In [ ]:
#load data
